In [1]:
import numpy as np
import pandas as pd

In [3]:
file_stats = pd.read_csv('./file_stats_sentiment.csv')
file_stats.head()

,Id,Sex,Age,Industry,Constellation,sentiment,Sentiment
0,1000331,female,37,NaN,Leo,0.151130,0.151130
1,1000866,female,17,Student,Libra,0.087342,0.087342
2,1004904,male,23,Arts,Capricorn,0.051705,0.051705
3,1005076,female,25,Arts,Cancer,0.016679,0.016679
4,1005545,male,25,Engineering,Sagittarius,0.123000,0.123000
